In [1]:
import pandas as pd
from pathlib import Path

### Path

In [39]:
EC35_DIR = Path.cwd().parent / "multimodal_er" / "EmoComics35"
DATA_FILES_DIR = EC35_DIR / "data_files"

In [40]:
DATA_FILE = DATA_FILES_DIR / "emocomics35.csv"

### Pandas DataFrame

In [41]:
df = pd.read_csv(DATA_FILE, index_col=0)

In [42]:
df

,Source text in English,Number of words,Signs with spaces,Signs without spaces,Translated text in Spanish; Castilian,Number of words.1,Signs with spaces.1,Signs without spaces.1,Page,Panel,Balloon,Annotations,source_file,file_nr,speaker,raw_emotions,emotions,split,comicbook_title
0,THIS VILE THING ATTACKED THE SMALL BEASTS OF M...,10,56,46,Esa cosa vil ha atacado a las bestezuelas de m...,11,58,48,1,1,2,2024-08-26 - aselermekova20\nFeeling:AN5-DI0-F...,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,1499,AQUANYX,AN5-DI0-FE0-SA0-SU0-JO0,['Anger'],TRAIN,Jurassic League #4
1,… IT PUNCHED MY BEAUTIFUL MATILDA… AND NOW IT ...,12,60,49,...Ha golpeado a mi preciosa Matilda... y ahor...,12,68,57,1,1,3,2024-08-26 - aselermekova20\nFeeling:AN5-DI0-F...,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,1499,AQUANYX,AN5-DI0-FE0-SA0-SU0-JO0,['Anger'],TRAIN,Jurassic League #4
2,MY MASTER!,2,10,9,¡Amo mío!,2,9,8,1,1,4,2024-08-26 - aselermekova20\nFeeling:AN0-DI0-F...,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,1499,ID-1,AN0-DI0-FE5-SA0-SU0-JO0,['Fear'],TRAIN,Jurassic League #4
3,PLEASE!,1,7,7,¡Por favor!,2,11,10,1,2,1,2024-08-26 - aselermekova20\nFeeling:AN0-DI0-F...,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,1499,ID-1,AN0-DI0-FE5-SA0-SU0-JO0,['Fear'],TRAIN,Jurassic League #4
4,BUT I HAVE NOT CHASED THIS MONSTER ALL THIS WA...,14,65,52,¡Pero no he perseguido a ese monstruo hasta aq...,14,74,61,1,3,1,2024-08-26 - aselermekova20\nFeeling:AN0-DI0-F...,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,1499,AQUANYX,AN0-DI0-FE3-SA5-SU0-JO0,"['Fear', 'Sadness']",TRAIN,Jurassic League #4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7124,SO WHY NOT MAN-ALTERED BORDERS?,5,31,27,"Así pues, ¿por qué no alterarlas?",6,33,28,19,5,2,2024-08-02 - aidaraliev12345\nFeeling:AN0-DI0-...,QC copy - 2200 - Stillwater 13.xlsx,2200,GALEN,AN0-DI0-FE0-SA0-SU0-JO1,['Joy'],TEST,Stillwater #13
7125,"I HAVE AN IDEA, GENTLEMEN. ONE THAT WILL ALLEV...",19,105,87,"Caballeros, se me ha ocurrido una idea que pal...",22,121,100,19,5,3,2024-08-02 - aidaraliev12345\nFeeling:AN0-DI0-...,QC copy - 2200 - Stillwater 13.xlsx,2200,GALEN,AN0-DI0-FE0-SA0-SU0-JO1,['Joy'],TEST,Stillwater #13
7126,LET'S EXPAND OUR WORLD SOME. THE MAP IS OF THE...,11,54,44,Ampliemos un poco nuestro mundo. Es un mapa de...,10,56,47,19,6,1,2024-08-02 - aidaraliev12345\nFeeling:AN0-DI0-...,QC copy - 2200 - Stillwater 13.xlsx,2200,GALEN,AN0-DI0-FE0-SA0-SU0-JO2,['Joy'],TEST,Stillwater #13
7127,SO LET'S CHANGE OUR BORDERS…,5,28,24,Cambiemos las fronteras...,3,26,24,19,6,2,2024-08-02 - aidaraliev12345\nFeeling:AN0-DI0-...,QC copy - 2200 - Stillwater 13.xlsx,2200,GALEN,AN0-DI0-FE0-SA0-SU0-JO2,['Joy'],TEST,Stillwater #13


In [43]:
df.columns

Index(['Source text in English', 'Number of words', 'Signs with spaces',
       'Signs without spaces', 'Translated text in Spanish; Castilian',
       'Number of words.1', 'Signs with spaces.1', 'Signs without spaces.1',
       'Page', 'Panel', 'Balloon', 'Annotations', 'source_file', 'file_nr',
       'speaker', 'raw_emotions', 'emotions', 'split', 'comicbook_title'],
      dtype='object')

In [44]:
columns = ['Source text in English', 'Page', 'Panel', 'Balloon', 'source_file', 'file_nr', 'speaker', 'emotions', 'split', 'comicbook_title']

In [45]:
df = df[columns]

In [46]:
df

,Source text in English,Page,Panel,Balloon,source_file,file_nr,speaker,emotions,split,comicbook_title
0,THIS VILE THING ATTACKED THE SMALL BEASTS OF M...,1,1,2,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,1499,AQUANYX,['Anger'],TRAIN,Jurassic League #4
1,… IT PUNCHED MY BEAUTIFUL MATILDA… AND NOW IT ...,1,1,3,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,1499,AQUANYX,['Anger'],TRAIN,Jurassic League #4
2,MY MASTER!,1,1,4,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,1499,ID-1,['Fear'],TRAIN,Jurassic League #4
3,PLEASE!,1,2,1,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,1499,ID-1,['Fear'],TRAIN,Jurassic League #4
4,BUT I HAVE NOT CHASED THIS MONSTER ALL THIS WA...,1,3,1,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,1499,AQUANYX,"['Fear', 'Sadness']",TRAIN,Jurassic League #4
...,...,...,...,...,...,...,...,...,...,...
7124,SO WHY NOT MAN-ALTERED BORDERS?,19,5,2,QC copy - 2200 - Stillwater 13.xlsx,2200,GALEN,['Joy'],TEST,Stillwater #13
7125,"I HAVE AN IDEA, GENTLEMEN. ONE THAT WILL ALLEV...",19,5,3,QC copy - 2200 - Stillwater 13.xlsx,2200,GALEN,['Joy'],TEST,Stillwater #13
7126,LET'S EXPAND OUR WORLD SOME. THE MAP IS OF THE...,19,6,1,QC copy - 2200 - Stillwater 13.xlsx,2200,GALEN,['Joy'],TEST,Stillwater #13
7127,SO LET'S CHANGE OUR BORDERS…,19,6,2,QC copy - 2200 - Stillwater 13.xlsx,2200,GALEN,['Joy'],TEST,Stillwater #13


In [47]:
df.columns = ['Utterance', 'Page', 'Panel', 'Balloon', 'Source_file', 'File_nr', 'Speaker', 'Emotions', 'Split', 'Comicbook_title']

In [48]:
df

,Utterance,Page,Panel,Balloon,Source_file,File_nr,Speaker,Emotions,Split,Comicbook_title
0,THIS VILE THING ATTACKED THE SMALL BEASTS OF M...,1,1,2,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,1499,AQUANYX,['Anger'],TRAIN,Jurassic League #4
1,… IT PUNCHED MY BEAUTIFUL MATILDA… AND NOW IT ...,1,1,3,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,1499,AQUANYX,['Anger'],TRAIN,Jurassic League #4
2,MY MASTER!,1,1,4,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,1499,ID-1,['Fear'],TRAIN,Jurassic League #4
3,PLEASE!,1,2,1,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,1499,ID-1,['Fear'],TRAIN,Jurassic League #4
4,BUT I HAVE NOT CHASED THIS MONSTER ALL THIS WA...,1,3,1,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,1499,AQUANYX,"['Fear', 'Sadness']",TRAIN,Jurassic League #4
...,...,...,...,...,...,...,...,...,...,...
7124,SO WHY NOT MAN-ALTERED BORDERS?,19,5,2,QC copy - 2200 - Stillwater 13.xlsx,2200,GALEN,['Joy'],TEST,Stillwater #13
7125,"I HAVE AN IDEA, GENTLEMEN. ONE THAT WILL ALLEV...",19,5,3,QC copy - 2200 - Stillwater 13.xlsx,2200,GALEN,['Joy'],TEST,Stillwater #13
7126,LET'S EXPAND OUR WORLD SOME. THE MAP IS OF THE...,19,6,1,QC copy - 2200 - Stillwater 13.xlsx,2200,GALEN,['Joy'],TEST,Stillwater #13
7127,SO LET'S CHANGE OUR BORDERS…,19,6,2,QC copy - 2200 - Stillwater 13.xlsx,2200,GALEN,['Joy'],TEST,Stillwater #13


### Page Grouping

In [65]:
page_df = df.groupby(["Source_file", "Page"]).agg({
    
    "Utterance": list,
    "Speaker": list,
    "Emotions": list,
    "Panel": list,
    "Balloon": list,
    "File_nr": list,
    "Split": list,
    "Comicbook_title": list
}).reset_index()

In [66]:
page_df

,Source_file,Page,Utterance,Speaker,Emotions,Panel,Balloon,File_nr,Split,Comicbook_title
0,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,1,[THIS VILE THING ATTACKED THE SMALL BEASTS OF ...,"[AQUANYX, AQUANYX, ID-1, ID-1, AQUANYX, ID-1, ...","[['Anger'], ['Anger'], ['Fear'], ['Fear'], ['F...","[1, 1, 1, 2, 3, 3, 3, 4, 5, 6]","[2, 3, 4, 1, 1, 2, 3, 1, 2, 1]","[1499, 1499, 1499, 1499, 1499, 1499, 1499, 149...","[TRAIN, TRAIN, TRAIN, TRAIN, TRAIN, TRAIN, TRA...","[Jurassic League #4, Jurassic League #4, Juras..."
1,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,2,"[NO-- #GKKK…#, #CHOMP!, BY THE SKIN OF MATILD...","[ID-1, BLACKMANTASAURUS, AQUANYX, AQUANYX, BLA...","[['Fear'], ['Anger'], ['Surprise'], ['Anger'],...","[1, 1, 2, 3, 3, 3, 3, 3, 3]","[1, 2, 1, 1, 2, 3, 5, 6, 7]","[1499, 1499, 1499, 1499, 1499, 1499, 1499, 149...","[TRAIN, TRAIN, TRAIN, TRAIN, TRAIN, TRAIN, TRA...","[Jurassic League #4, Jurassic League #4, Juras..."
2,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,3,"[COME ON, BEAST!, SHOW YOURSELF!, WHY DO YOU H...","[AQUANYX, AQUANYX, AQUANYX, AQUANYX]","[['Joy'], ['Joy'], ['Anger'], ['Anger']]","[1, 1, 1, 1]","[1, 2, 5, 6]","[1499, 1499, 1499, 1499]","[TRAIN, TRAIN, TRAIN, TRAIN]","[Jurassic League #4, Jurassic League #4, Juras..."
3,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,4,[#AARGH! ],[AQUANYX],"[['Fear', 'Surprise']]",[2],[2],[1499],[TRAIN],[Jurassic League #4]
4,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,5,"[I, THE GREEN TORCH, HAVE BEEN TASKED WITH PRO...","[GREEN TORCH, GREEN TORCH, ATROCITAURUS, ATROC...","[['Anger'], ['Anger'], ['Fear'], ['Fear', 'Sur...","[1, 1, 1, 3, 4, 5]","[2, 3, 5, 1, 1, 2]","[1499, 1499, 1499, 1499, 1499, 1499]","[TRAIN, TRAIN, TRAIN, TRAIN, TRAIN, TRAIN]","[Jurassic League #4, Jurassic League #4, Juras..."
...,...,...,...,...,...,...,...,...,...,...
869,QC copy - 2200 - Stillwater 13.xlsx,16,[WE WERE IN GALEN'S OFFICE. YOU WERE ABOUT TO ...,"[LAURA, LAURA, LAURA, DANIEL, DANIEL, DANIEL, ...","[['Anger'], ['Anger'], ['Anger'], ['Anger'], [...","[1, 1, 1, 2, 2, 3, 3, 3, 3, 4, 4, 5, 5, 6]","[1, 2, 3, 1, 2, 2, 3, 4, 5, 1, 2, 1, 2, 1]","[2200, 2200, 2200, 2200, 2200, 2200, 2200, 220...","[TEST, TEST, TEST, TEST, TEST, TEST, TEST, TES...","[Stillwater #13, Stillwater #13, Stillwater #1..."
870,QC copy - 2200 - Stillwater 13.xlsx,17,"[SO WHAT ARE WE GOING TO DO?, THE WAY I SEE IT...","[ID-6, GALEN, ID-7, GALEN, GALEN, GALEN, GALEN]","[['Sadness', 'Surprise'], ['Anger'], ['Anger']...","[3, 3, 3, 3, 4, 4, 5]","[1, 2, 3, 4, 1, 2, 1]","[2200, 2200, 2200, 2200, 2200, 2200, 2200]","[TEST, TEST, TEST, TEST, TEST, TEST, TEST]","[Stillwater #13, Stillwater #13, Stillwater #1..."
871,QC copy - 2200 - Stillwater 13.xlsx,18,"[KIDDIE COUNCIL'S BEEN GOING A LONG TIME... , ...","[TED, KREEGS, ID-8, ID-8, GALEN, GALEN, KREEGS...","[['Anger', 'Sadness'], ['Anger'], ['Anger'], [...","[1, 1, 1, 2, 3, 4, 5, 6, 6, 7, 7]","[1, 2, 3, 1, 1, 1, 1, 1, 2, 1, 2]","[2200, 2200, 2200, 2200, 2200, 2200, 2200, 220...","[TEST, TEST, TEST, TEST, TEST, TEST, TEST, TES...","[Stillwater #13, Stillwater #13, Stillwater #1..."
872,QC copy - 2200 - Stillwater 13.xlsx,19,[IT'S BEEN… PEACEFUL. ASIDE FROM SHIT LIKE THI...,"[KREEGS, GALEN, GALEN, KREEGS, GALEN, GALEN, K...","[['Anger'], ['Joy'], ['Joy'], ['Anger', 'Surpr...","[1, 1, 1, 2, 2, 3, 4, 4, 5, 5, 6, 6]","[1, 2, 3, 1, 2, 1, 1, 2, 2, 3, 1, 2]","[2200, 2200, 2200, 2200, 2200, 2200, 2200, 220...","[TEST, TEST, TEST, TEST, TEST, TEST, TEST, TES...","[Stillwater #13, Stillwater #13, Stillwater #1..."


In [67]:
def clean_columns(row, col):
    
    return row[col][0]

In [68]:
page_df["File_nr"] = page_df.apply(lambda row: clean_columns(row, "File_nr"), axis=1)
page_df["Split"] = page_df.apply(lambda row: clean_columns(row, "Split"), axis=1)
page_df["Comicbook_title"] = page_df.apply(lambda row: clean_columns(row, "Comicbook_title"), axis=1)

In [69]:
page_df

,Source_file,Page,Utterance,Speaker,Emotions,Panel,Balloon,File_nr,Split,Comicbook_title
0,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,1,[THIS VILE THING ATTACKED THE SMALL BEASTS OF ...,"[AQUANYX, AQUANYX, ID-1, ID-1, AQUANYX, ID-1, ...","[['Anger'], ['Anger'], ['Fear'], ['Fear'], ['F...","[1, 1, 1, 2, 3, 3, 3, 4, 5, 6]","[2, 3, 4, 1, 1, 2, 3, 1, 2, 1]",1499,TRAIN,Jurassic League #4
1,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,2,"[NO-- #GKKK…#, #CHOMP!, BY THE SKIN OF MATILD...","[ID-1, BLACKMANTASAURUS, AQUANYX, AQUANYX, BLA...","[['Fear'], ['Anger'], ['Surprise'], ['Anger'],...","[1, 1, 2, 3, 3, 3, 3, 3, 3]","[1, 2, 1, 1, 2, 3, 5, 6, 7]",1499,TRAIN,Jurassic League #4
2,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,3,"[COME ON, BEAST!, SHOW YOURSELF!, WHY DO YOU H...","[AQUANYX, AQUANYX, AQUANYX, AQUANYX]","[['Joy'], ['Joy'], ['Anger'], ['Anger']]","[1, 1, 1, 1]","[1, 2, 5, 6]",1499,TRAIN,Jurassic League #4
3,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,4,[#AARGH! ],[AQUANYX],"[['Fear', 'Surprise']]",[2],[2],1499,TRAIN,Jurassic League #4
4,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,5,"[I, THE GREEN TORCH, HAVE BEEN TASKED WITH PRO...","[GREEN TORCH, GREEN TORCH, ATROCITAURUS, ATROC...","[['Anger'], ['Anger'], ['Fear'], ['Fear', 'Sur...","[1, 1, 1, 3, 4, 5]","[2, 3, 5, 1, 1, 2]",1499,TRAIN,Jurassic League #4
...,...,...,...,...,...,...,...,...,...,...
869,QC copy - 2200 - Stillwater 13.xlsx,16,[WE WERE IN GALEN'S OFFICE. YOU WERE ABOUT TO ...,"[LAURA, LAURA, LAURA, DANIEL, DANIEL, DANIEL, ...","[['Anger'], ['Anger'], ['Anger'], ['Anger'], [...","[1, 1, 1, 2, 2, 3, 3, 3, 3, 4, 4, 5, 5, 6]","[1, 2, 3, 1, 2, 2, 3, 4, 5, 1, 2, 1, 2, 1]",2200,TEST,Stillwater #13
870,QC copy - 2200 - Stillwater 13.xlsx,17,"[SO WHAT ARE WE GOING TO DO?, THE WAY I SEE IT...","[ID-6, GALEN, ID-7, GALEN, GALEN, GALEN, GALEN]","[['Sadness', 'Surprise'], ['Anger'], ['Anger']...","[3, 3, 3, 3, 4, 4, 5]","[1, 2, 3, 4, 1, 2, 1]",2200,TEST,Stillwater #13
871,QC copy - 2200 - Stillwater 13.xlsx,18,"[KIDDIE COUNCIL'S BEEN GOING A LONG TIME... , ...","[TED, KREEGS, ID-8, ID-8, GALEN, GALEN, KREEGS...","[['Anger', 'Sadness'], ['Anger'], ['Anger'], [...","[1, 1, 1, 2, 3, 4, 5, 6, 6, 7, 7]","[1, 2, 3, 1, 1, 1, 1, 1, 2, 1, 2]",2200,TEST,Stillwater #13
872,QC copy - 2200 - Stillwater 13.xlsx,19,[IT'S BEEN… PEACEFUL. ASIDE FROM SHIT LIKE THI...,"[KREEGS, GALEN, GALEN, KREEGS, GALEN, GALEN, K...","[['Anger'], ['Joy'], ['Joy'], ['Anger', 'Surpr...","[1, 1, 1, 2, 2, 3, 4, 4, 5, 5, 6, 6]","[1, 2, 3, 1, 2, 1, 1, 2, 2, 3, 1, 2]",2200,TEST,Stillwater #13


In [70]:
page_df.columns = ["SourceFile", "Page", "PageUtterances", "PageSpeakers", "PageEmotions", "PagePanels", "PageBalloons", "FileNr", "Split", "ComicBookTitle"]

In [76]:
def check_egalite(row):
    
    return 1 if (len(row.PageUtterances) == len(row.PageSpeakers) == len(row.PageEmotions) == len(row.PagePanels) == len(row.PageBalloons)) else 0

In [77]:
page_df["egalite_check"] = page_df.apply(lambda row: check_egalite(row), axis=1)

In [81]:
page_df["egalite_check"].value_counts()

egalite_check
1    874
Name: count, dtype: int64

In [83]:
page_df= page_df.drop(page_df.columns[-1], axis=1)

In [ ]:
#page_df.to_csv(DATA_FILES_DIR / "emocomics35_pg.csv")

### Add Images

In [85]:
pg_df = pd.read_csv(DATA_FILES_DIR / "emocomics35_pg.csv", index_col=0)

In [87]:
EC35_IMAGES_DIR = EC35_DIR / "raw_files" / "images"

In [93]:
def get_image_file_path(row):
    
    page_nr = row.Page
    comics_dir = "00" + str(row.FileNr)
    
    images_dir = Path(EC35_IMAGES_DIR) / comics_dir / "images"    
    file_name = "page" + f"{page_nr:05d}" + ".jpg"
        
    file_path = images_dir / file_name    
    
    return file_path

In [96]:
pg_df['image_path'] = pg_df.apply(lambda row: get_image_file_path(row), axis=1)

In [106]:
pg_df.to_csv(DATA_FILES_DIR / "emocomics35_pg_images.csv")